In [16]:
# pip install xlrd
# pip install pandas

In [17]:
# Tratamento de dados, ainda fora do ambiente kubernetes

import pandas as pd
import os

In [18]:
import pandas as pd

Descargas_EPB = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EPB.csv', delimiter=';')
Descargas_ERO = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ERO.csv', delimiter=';')
Descargas_EMT = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EMT.csv', delimiter=';')
Descargas_EMS = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EMS.csv', delimiter=';')
Descargas_EMR = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EMR.csv', delimiter=';')
Descargas_EAC = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EAC.csv', delimiter=';')
Descargas_ETO = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ETO.csv', delimiter=';')
Descargas_ESS = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ESS.csv', delimiter=';')
Descargas_ESE = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ESE.csv', delimiter=';')

Lista_Descargas = [Descargas_EPB, Descargas_ERO, Descargas_EMT, Descargas_EMS, Descargas_EMR, Descargas_EAC, Descargas_ETO, Descargas_ESS, Descargas_ESE]

Descargas_Energisa = pd.concat(Lista_Descargas, ignore_index=True)

Descargas_Energisa = Descargas_Energisa.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]

#  Excluir a primeira linha
Descargas_Energisa = Descargas_Energisa.drop(0)

# Renomear todas as colunas
novos_nomes_colunas = ['Localidade', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

Descargas_Energisa.columns = novos_nomes_colunas


In [19]:
Cod_IBGE = pd.read_excel(f'/home/ruanv/Documentos/cod_ibge.xls', header=6)
Ocorrencias = pd.read_csv(f'/home/ruanv/Documentos/ocorrencias-emergenciais-rede-distribuicao-2023.csv', delimiter=';', encoding='latin-1', low_memory=False)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [20]:
print(Ocorrencias)

        DatGeracaoConjuntoDados                                     NomAgente  \
0                    2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
1                    2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
2                    2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
3                    2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
4                    2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
...                         ...                                           ...   
6899052              2023-10-10                         Amazonas Energia S.A.   
6899053              2023-10-10                         Amazonas Energia S.A.   
6899054              2023-10-10                         Amazonas Energia S.A.   
6899055              2023-10-10                         Amazonas Energia S.A.   
6899056              2023-10-10                         Amazonas Energia S.A.   

            NumCPFCNPJ NumO

In [21]:
Ocorrencias = Ocorrencias[Ocorrencias['DscOcorrenciaAberta'].str.contains("DESCARGA ATMOSFERICA", case=False)]


In [22]:
Ocorrencias = pd.merge(Ocorrencias, Cod_IBGE, left_on='CodIBGE', right_on='Código Município Completo', how='left')

In [23]:
print(Ocorrencias)

       DatGeracaoConjuntoDados                                     NomAgente  \
0                   2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
1                   2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
2                   2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
3                   2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
4                   2023-10-10  ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A   
...                        ...                                           ...   
186494              2023-10-10                         Amazonas Energia S.A.   
186495              2023-10-10                         Amazonas Energia S.A.   
186496              2023-10-10                         Amazonas Energia S.A.   
186497              2023-10-10                         Amazonas Energia S.A.   
186498              2023-10-10                         Amazonas Energia S.A.   

           NumCPFCNPJ NumOcorrencia  Nu

In [24]:
print(Descargas_Energisa)

               Localidade Janeiro Fevereiro  Março Abril  Maio Junho Julho  \
1        Energisa Paraíba   28684     36175  18879  8429  4083     0     0   
2             Água Branca      45       104    119    19    27     0     0   
3                  Aguiar     322       524    721   166     2     0     0   
4           Alagoa Grande       3        19     41     3     0     0     0   
5             Alagoa Nova       6         2      0     2     0     0     0   
..                    ...     ...       ...    ...   ...   ...   ...   ...   
956         São Francisco       0         0      0     0     0     0     0   
957  São Miguel Do Aleixo       0         0      5     0     0     0     0   
958            Simão Dias       1        59     30     0     0     0     0   
959                Siriri       1        87     14     0     0     0     0   
960                 Telha       0         2      0     0     0     0     0   

    Agosto Setembro Outubro Novembro Dezembro  
1        1     